## Домашнее задание "Оценка точности модели, переобучение, регуляризация"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Загрузим данные из файла:

In [3]:
df = pd.read_csv('adult.csv',na_values='?')

In [4]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


Удалим незначимое поле fnlwgt

In [5]:
df.drop(['fnlwgt'],axis=1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
age                48842 non-null int64
workclass          46043 non-null object
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         46033 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     47985 non-null object
income             48842 non-null object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


Данный dataset содержит три категории с пропущенными значениями: workclass, occupation, native-country        

Рассмотрим сколько уникальных значений содержится в каждой категориальной фичи, если не очень много переведем их фичи методом get_dummies

In [7]:
df.workclass.value_counts(dropna=False)

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
NaN                  2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

Заполним пропущенные значения наиболее часто встречающимся значением Private

In [8]:
df['workclass'] = df['workclass'].fillna(value='Private')

In [9]:
df.education.value_counts(dropna=False)

HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: education, dtype: int64

In [10]:
df.occupation.value_counts(dropna=False)

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
NaN                  2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [11]:
df.columns=['age', 'workclass', 'education', 'educational_num',
       'marital_status', 'occupation', 'relationship', 'race', 'gender',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income']

In [12]:
df.marital_status.value_counts(dropna=False)

Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: marital_status, dtype: int64

In [13]:
df.relationship.value_counts(dropna=False)

Husband           19716
Not-in-family     12583
Own-child          7581
Unmarried          5125
Wife               2331
Other-relative     1506
Name: relationship, dtype: int64

In [14]:
df.race.value_counts(dropna=False)

White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64

In [15]:
df.gender.value_counts(dropna=False)

Male      32650
Female    16192
Name: gender, dtype: int64

In [16]:
df.native_country.value_counts(dropna=False)

United-States                 43832
Mexico                          951
NaN                             857
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                        

Заполним пропущенные значения и наименее часто встречающиеся значением Other

In [17]:
df['country']=df.apply(lambda row: row.native_country if row.native_country in ['United-States', 
       'Mexico', 'Philippines', 'Germany', 'Puerto-Rico',
       'Canada', 'El-Salvador', 'India', 'Cuba', 'England', 'China', 'South',
       'Jamaica', 'Italy', 'Dominican-Republic'] else 'Other',axis=1)
df.drop('native_country',axis=1,inplace=True)

In [18]:
df.country.value_counts()

United-States         43832
Other                  2070
Mexico                  951
Philippines             295
Germany                 206
Puerto-Rico             184
Canada                  182
El-Salvador             155
India                   151
Cuba                    138
England                 127
China                   122
South                   115
Jamaica                 106
Italy                   105
Dominican-Republic      103
Name: country, dtype: int64

In [19]:
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df.drop(each,inplace=True,axis=1)
    return df

In [20]:
X=get_one_hot(df,['gender','race','relationship','marital_status','country','workclass','education'])
Y=X.income
X.drop(['occupation','income'],axis=1,inplace=True)


In [21]:
X.head()

,age,educational_num,capital_gain,capital_loss,hours_per_week,gender_Female,gender_Male,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,25,7,0,0,40,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,38,9,0,0,50,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,28,12,0,0,40,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,44,10,7688,0,40,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,18,10,0,0,30,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
X.describe()

,age,educational_num,capital_gain,capital_loss,hours_per_week,gender_Female,gender_Male,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,...,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,10.078089,1079.067626,87.502314,40.422382,0.331518,0.668482,0.009623,0.031100,0.095922,...,0.015478,0.032779,0.042197,0.164305,0.012162,0.323164,0.054400,0.001699,0.017075,0.222718
std,13.710510,2.570973,7452.019058,403.004552,12.391444,0.470764,0.470764,0.097624,0.173591,0.294487,...,0.123447,0.178060,0.201041,0.370556,0.109608,0.467690,0.226807,0.041189,0.129554,0.416075
min,17.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Сделаем масштабирование для всех фичей, т.к. логитическая регрессия чувствительна к различным масштабам.

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
scaler = StandardScaler()
X_scaled=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

In [25]:
X_scaled.head()

,age,educational_num,capital_gain,capital_loss,hours_per_week,gender_Female,gender_Male,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,-0.995129,-1.197259,-0.144804,-0.217127,-0.034087,-0.70422,0.70422,-0.098572,-0.179161,3.070047,...,-0.125387,-0.184093,-0.209896,-0.443406,-0.110957,-0.690988,-0.239853,-0.041258,-0.131803,-0.535289
1,-0.046942,-0.419335,-0.144804,-0.217127,0.772930,-0.70422,0.70422,-0.098572,-0.179161,-0.325728,...,-0.125387,-0.184093,-0.209896,-0.443406,-0.110957,1.447204,-0.239853,-0.041258,-0.131803,-0.535289
2,-0.776316,0.747550,-0.144804,-0.217127,-0.034087,-0.70422,0.70422,-0.098572,-0.179161,-0.325728,...,-0.125387,5.432051,-0.209896,-0.443406,-0.110957,-0.690988,-0.239853,-0.041258,-0.131803,-0.535289
3,0.390683,-0.030373,0.886874,-0.217127,-0.034087,-0.70422,0.70422,-0.098572,-0.179161,3.070047,...,-0.125387,-0.184093,-0.209896,-0.443406,-0.110957,-0.690988,-0.239853,-0.041258,-0.131803,1.868149
4,-1.505691,-0.030373,-0.144804,-0.217127,-0.841104,1.42001,-1.42001,-0.098572,-0.179161,-0.325728,...,-0.125387,-0.184093,-0.209896,-0.443406,-0.110957,-0.690988,-0.239853,-0.041258,-0.131803,1.868149


Попробуем получить и улучшить значения AUC для двух моделей LogisticRegression и RandomForestClassifier

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, accuracy_score, average_precision_score, roc_curve, roc_auc_score, auc
from sklearn.model_selection import GridSearchCV

In [27]:
le=LabelEncoder()
le.fit(Y)

LabelEncoder()

In [28]:
Y = le.transform(Y)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2)

## Логистическая регрессия

In [58]:
LogReg = LogisticRegression()

In [59]:
LogReg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [60]:
predictions_proba = LogReg.predict_proba(x_test)
predictions = LogReg.predict(x_test)
fpr, tpr, thres = roc_curve( y_test, predictions )
roc_auc_score( y_test, predictions)
auc(fpr, tpr)

0.74999791792636883

In [61]:
accuracy_score( y_test, predictions)

0.84276793940014327

Попробуем изменить гиперпараметры алгоритма логистической регресии, возможно это поможит улучшить качество алгоритма.

In [49]:
grid ={"penalty": ['l1','l2'],"C":[0.001,0.01,0.1,1,2,3]}
model_grid = GridSearchCV(LogisticRegression(),grid,cv=5,scoring='accuracy')

In [50]:
model_grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [52]:
model_grid.best_params_

{'C': 0.01, 'penalty': 'l1'}

In [53]:
predictions = model_grid.best_estimator_.predict(x_test)
fpr, tpr, thres = roc_curve( y_test, predictions )
roc_auc_score(y_test, predictions)
auc(fpr, tpr)

0.74743081366774933

In [56]:
model_grid.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
accuracy_score( y_test, predictions)

0.84440577336472511

C помощью подбора гиперпараметров нам незначительно удалось улучшить значение точности. Попробуем другой алгоритм

## Random Forest 

In [62]:
clf_rndforest = RandomForestClassifier()
clf_rndforest.fit(x_train,y_train)
predictions = clf_rndforest.predict(x_test)
fpr, tpr, thres = roc_curve( y_test, predictions )
roc_auc_score( y_test, predictions)
auc(fpr, tpr)

0.75262113105084816

In [63]:
accuracy_score( y_test, predictions)

0.83826389599754325

Попробуем улучшить значения с помощью подбора гиперпараметров:

In [71]:
bootstrap = [True, False]
max_depth = range(3,10)
max_features = ['auto', 'sqrt']
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 5)]

grid ={'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,               
               'bootstrap': bootstrap}

model_grid = GridSearchCV(RandomForestClassifier(),grid,cv=5,scoring='roc_auc',n_jobs=-1)
model_grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 40, 50], 'max_features': ['auto', 'sqrt'], 'max_depth': range(3, 10), 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [72]:
model_grid.best_params_

{'bootstrap': False,
 'max_depth': 9,
 'max_features': 'auto',
 'n_estimators': 40}

In [73]:
predictions = model_grid.best_estimator_.predict(x_test)
fpr, tpr, thres = roc_curve( y_test, predictions )
roc_auc_score(y_test, predictions)
auc(fpr, tpr)

0.74498833468335368

In [74]:
accuracy_score( y_test, predictions)

0.85300440167877978

С помощью GridSearchCV нам удалось улучшить показатель точности